# Cleaning and EDA:

I'll start off by doing some cleaning to our data set and fixing any data type issues we may have. Then I'll conduct some exploratory data analysis to find trends/patterns in the data

In [1]:
#imports

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_style('whitegrid')


#options

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.set_option('chained_assignment', None)


In [2]:
#read in train data
train = pd.read_csv('./datasets/train.csv')
train.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,NaN,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,SBrkr,744,700,0,1444,0.0,0.0,2,1,3,1,TA,7,Typ,0,NaN,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,GasA,TA,Y,SBrkr,831,614,0,1445,0.0,0.0,2,0,3,1,TA,6,Typ,0,NaN,Detchd,1957.0,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [3]:
#checking data types
train.Alley.isnull().sum()

1911

In [4]:
#checking for null values

#thanks to Chanakya Vivek Kapoor on Kaggle for this solution 

def null_values(df):
    
    columns = df.isnull().sum().index
    values = df.isnull().sum().values
    null_percentage = []
    
    #finding null values in each column, dividing by shape of the data frame and multiplying by 100
    [null_percentage.append((df[column]).isnull().sum()/ df[column].shape[0] * 100) for column in columns]
    
    null_dict = {'Columns':columns, 'Null Values': values, 'Percentage of Null Values': null_percentage}
    
    null_table = pd.DataFrame(data = null_dict)
    
    return null_table

In [5]:
null_table = null_values(train)
null_table.sort_values(by = 'Percentage of Null Values', ascending = False).head(15)

,Columns,Null Values,Percentage of Null Values
73,Pool QC,2042,99.561190
75,Misc Feature,1986,96.830814
7,Alley,1911,93.174061
74,Fence,1651,80.497318
58,Fireplace Qu,1000,48.756704
4,Lot Frontage,330,16.089712
61,Garage Finish,114,5.558264
64,Garage Qual,114,5.558264
60,Garage Yr Blt,114,5.558264
65,Garage Cond,114,5.558264


## Dealing with missing values:

From our results above, 5 of our columns have about or over 50% of it's data missing.

It's important to note that a column having null values could mean that data was not recorded or does not exist(ie: NA in the 'Pool QC' column means the property has no pool) so it can still be valuable information.

For the purposes of this regression model, I will get rid of columns that contain more than 90% of null data **'Pool QC', 'Alley' and 'Misc Feature'**. It would mean that data is truly missing or these properties do not contain a misc feature, a pool or an alley which is not helping much from an analysis point of view.

## Ordinal and Nominal data:
Some of our data contains values that are ordinal, meaning that the values have some intrinsic order. This means they can be ranked from best to worst or numerically (from 1-5 for example). Converting our ordinal data columns would be best in order to assess their impact.
Nominal values have no intrinsic order and are typically descriptive. We will convert these to dummy columns later on.

In [6]:
def df(dataset):
    #remove columns with more than 90% of null values.
    train_data = dataset.loc[:, dataset.isin([np.nan]).mean() < .90]
    
    #fill in null values for columns with missing data that I want to keep
    train_data.fillna({'Fireplace Qu': 'NA', 'Fence': 'NA', 'Lot Frontage': 0}, inplace = True)
    
    #remaping ordinal columns to numerical values
    train_data['Exter Qual'] = train_data['Exter Qual'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1})
    train_data['Exter Cond'] = train_data['Exter Cond'].map({'Ex':5, 'Gd': 4, 'TA':3, 'Fa':2, 'Po':1})
    train_data['Bsmt Qual']= train_data['Bsmt Qual'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA': 0})
    train_data['Bsmt Cond'] = train_data['Bsmt Cond'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po':1, 'NA': 0})
    train_data['Bsmt Exposure'] = train_data['Bsmt Exposure'].map({'Gd': 4, 'Av': 3, 'Mn': 2, 'No':1, 'NA': 0})
    train_data['BsmtFin Type 1'] = train_data['BsmtFin Type 1'].map({'GLQ': 6, 'ALQ':5, 'BLQ':4, 'Rec':3, 'LwQ':2, 'Unf':1, 'NA':0})
    train_data['BsmtFin Type 2'] = train_data['BsmtFin Type 2'].map({'GLQ': 6, 'ALQ':5, 'BLQ': 4, 'Rec':3, 'LwQ':2, 'Unf':1, 'NA':0})
    train_data['Heating QC'] = train_data['Heating QC'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa':2, 'Po':1})
    train_data['Electrical'] = train_data['Electrical'].map({'SBrkr':4, 'FuseA':3, "FuseF":2, 'FuseP':1, 'Mix':0})
    train_data['Kitchen Qual']= train_data['Kitchen Qual'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1})
    train_data['Functional']= train_data['Functional'].map({'Typ':7, 'Min1':6, 'Min2':5, 'Mod':4, 'Maj1':3, 'Maj2':2, 'Sev':1, 'Sal':0 })
    train_data['Fireplace Qu'] = train_data['Fireplace Qu'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0})
    train_data['Garage Finish'] = train_data['Garage Finish'].map({'Fin':3, 'RFn':2, 'Unf':1, 'NA':0 })
    train_data['Garage Qual'] = train_data['Garage Qual'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0})
    train_data['Garage Cond']= train_data['Garage Cond'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0})
    train_data['Paved Drive']= train_data['Paved Drive'].map({'Y':3, 'P':2, 'N':1})
    train_data['Fence']= train_data['Fence'].map({'GdPrv':4, 'MnPrv':3, 'GdWo':2, 'MnWw':1, 'NA':0})
    
#     #dummify left over categorical data columns
#     train_data = pd.get_dummies(data = train_data, columns = list(train_data.select_dtypes(['object']).columns), drop_first=True)
    
    #drop rest of null values less than 90%
    train_data.dropna(inplace=True)
    
    return train_data

In [7]:
train = df(train)
train.isnull().sum()

Id                 0
PID                0
MS SubClass        0
MS Zoning          0
Lot Frontage       0
Lot Area           0
Street             0
Lot Shape          0
Land Contour       0
Utilities          0
Lot Config         0
Land Slope         0
Neighborhood       0
Condition 1        0
Condition 2        0
Bldg Type          0
House Style        0
Overall Qual       0
Overall Cond       0
Year Built         0
Year Remod/Add     0
Roof Style         0
Roof Matl          0
Exterior 1st       0
Exterior 2nd       0
Mas Vnr Type       0
Mas Vnr Area       0
Exter Qual         0
Exter Cond         0
Foundation         0
Bsmt Qual          0
Bsmt Cond          0
Bsmt Exposure      0
BsmtFin Type 1     0
BsmtFin SF 1       0
BsmtFin Type 2     0
BsmtFin SF 2       0
Bsmt Unf SF        0
Total Bsmt SF      0
Heating            0
Heating QC         0
Central Air        0
Electrical         0
1st Flr SF         0
2nd Flr SF         0
Low Qual Fin SF    0
Gr Liv Area        0
Bsmt Full Bat

In [9]:
train['Mo Sold'].astype(str)

0        3
1        4
2        1
3        4
4        3
5        6
6        6
7        5
8        6
9        9
10       7
11       2
13      12
14       6
15       3
16       5
17       6
18       4
19       6
20       5
21      10
23       3
24       1
25       4
26       4
27       7
29       7
30       7
31       6
32       4
33       4
34       9
35       4
36       7
37       1
38       9
39       7
40       6
42       7
43      11
44       2
45       6
46       7
47       2
48       2
49      10
50       6
51      12
52       7
54       6
55       3
56       2
57       3
58       6
59      11
60       4
61       8
62       4
63       6
64       7
66       6
67       4
68       6
69       4
70       1
71       9
72       7
73       4
74       4
75       7
76       4
77       5
78       1
80       6
81       7
82       6
83       7
84       4
85       6
87       6
88       8
89       4
90       5
91       6
92      10
94       4
95       2
96       3
97       1
98       8
99       6